In [77]:
import re
import pandas as pd
import numpy as np

In [313]:
df1 = pd.read_csv('P20190100135_1/proteinGroups.txt', sep='\t')
df2 = pd.read_csv('P20190100135_2/proteinGroups.txt', sep='\t')
df3 = pd.read_csv('P20190100135_3/proteinGroups.txt', sep='\t')
df4 = pd.read_csv('P20190100135_4/proteinGroups.txt', sep='\t')
df5 = pd.read_csv('P20190100135_5/proteinGroups.txt', sep='\t')
df1.tail()

,Protein IDs,Majority protein IDs,Peptide counts (all),Peptide counts (razor+unique),Peptide counts (unique),Protein names,Gene names,Fasta headers,Number of proteins,Peptides,...,Potential contaminant,id,Peptide IDs,Peptide is razor,Mod. peptide IDs,Evidence IDs,MS/MS IDs,Best MS/MS,Oxidation (M) site IDs,Oxidation (M) site positions
6474,REV__Q9Y2G9,REV__Q9Y2G9,1,1,1,NaN,NaN,sp|Q9Y2G9|SBNO2_HUMAN Protein strawberry notch...,1,1,...,NaN,6474,18622,True,20309,903358;903359;903360;903361;903362;903363;9033...,641865;641866;641867;641868;641869;641870;6418...,641865,NaN,NaN
6475,REV__Q9Y5C1,REV__Q9Y5C1,3,3,3,NaN,NaN,sp|Q9Y5C1|ANGL3_HUMAN Angiopoietin-related pro...,1,3,...,NaN,6475,26574;47715;50356,True;True;True,28956;52674;55516,1308785;1308786;1308787;2342767;2342768;234276...,913884;1589165;1589166;1589167;1589168;1589169...,913884;1589176;1655324,NaN,NaN
6476,REV__Q9Y5T5,REV__Q9Y5T5,3,3,3,NaN,NaN,sp|Q9Y5T5|UBP16_HUMAN Ubiquitin carboxyl-termi...,1,3,...,NaN,6476,35178;42484;60962,True;True;True,38359;47012;67190,1756910;1756911;1756912;1756913;1756914;175691...,1212178;1212179;1212180;1445218;1445219;206687...,1212178;1445218;2066884,NaN,NaN
6477,REV__Q9Y6D6,REV__Q9Y6D6,3,3,3,NaN,NaN,sp|Q9Y6D6|BIG1_HUMAN Brefeldin A-inhibited gua...,1,3,...,NaN,6477,9856;32562;44840,True;True;True,10794;35480;49572,470716;470717;470718;470719;470720;470721;4707...,328428;1122219;1122220;1122221;1122222;1122223...,328428;1122225;1520906,NaN,NaN
6478,REV__Q9Y6R7,REV__Q9Y6R7,4,4,4,NaN,NaN,sp|Q9Y6R7|FCGBP_HUMAN IgGFc-binding protein OS...,1,4,...,NaN,6478,23425;35002;40250;40588,True;True;True;True,25540;38119;44560;44935,1139201;1139202;1139203;1139204;1139205;113920...,799649;799650;1201503;1201504;1201505;1376964;...,799649;1201504;1376964;1387409,NaN,NaN


In [314]:
def clean(df, index):
    # drop_list
    columns = df.columns.tolist()
    intensity_list = [i for i in columns[columns.index('Intensity'):] if re.search('Intensity|iBAQ', i, re.I)]
    newcolumns = columns[:2]
    newcolumns.extend(intensity_list)
    # 对 Majority protein IDs 列转成多行
    df = df[~df['Majority protein IDs'].str.contains('CON|REV', regex=True)]
    df = df.drop('Majority protein IDs', axis=1).join(df['Majority protein IDs'].str.split(';', expand=True).stack().reset_index(level=1, drop=True).rename('Majority protein IDs'))
#     df.index = df['Majority protein IDs']
    if index != 1:
        df = df[newcolumns]
        df.drop(columns[0],axis=1, inplace=True)
    else:
        df = df[columns[:columns.index(intensity_list[-1]) + 1]]
    return df

df1 = clean(df1, 1)
df2 = clean(df2, 2)
df3 = clean(df3, 3)
df4 = clean(df4, 4)
df5 = clean(df5, 5)

In [310]:
df_merge1 = pd.merge(df1, df2, how='outer', on='Majority protein IDs', suffixes=('_1', '_2'))
df_merge2 = pd.merge(df_merge1, df3, how='outer', on='Majority protein IDs')
df_merge3 = pd.merge(df_merge2, df4, how='outer', on='Majority protein IDs', suffixes=('_3', '_4'))
df_merge4 = pd.merge(df_merge3, df5, how='outer', on='Majority protein IDs')
df_merge = df_merge4.fillna(0)
# df_merge.head()
# df_merge.to_csv('test.csv', index=False)

In [316]:
def ab(df):
    return';'.join(df.values)
newcolumns = df_merge.columns.tolist()
newcolumns.remove('Majority protein IDs')
newdf = df_merge.groupby(newcolumns)['Majority protein IDs'].apply(ab)  ## 多行合并一行
newdf = newdf.reset_index()
maj_id = newdf['Majority protein IDs']
newdf.drop('Majority protein IDs', axis=1, inplace=True)
newdf.insert(1, 'Majority protein IDs', maj_id)
newdf.replace(0, '', inplace=True)

newdf.to_excel('protein_merge.xlsx', index=False)
# len(newcolumns)

In [291]:
t1 = df_merge.iloc[23,:]
t2 = df_merge.iloc[24,:]
t1.drop('Majority protein IDs', inplace=True)

t2.drop('Majority protein IDs', inplace=True)
# t2 = t2.fillna(0)

t2['LFQ intensity A1-12']
# t1 == t2


# total = {i for i in df_merge4['Majority protein IDs']}

# pro_1 = {i for i in df1['Majority protein IDs']}

# len(total)

# # len(total & pro_1), len(pro1)

d:\python\lib\site-packages\pandas\core\generic.py:3114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(obj)


0.0

In [265]:
df5.head()

,Majority protein IDs,Intensity,Intensity A1-1,Intensity A1-12,Intensity A1-16,Intensity A1-19,Intensity A1-2,Intensity A2-16,Intensity A2-17,Intensity A2-18,...,LFQ intensity C2-18,LFQ intensity C2-8,LFQ intensity C2-9,LFQ intensity QC_20190614162630,LFQ intensity QC_20190619140322,LFQ intensity QC_20190621214931,LFQ intensity QC_20190629114230,LFQ intensity QC_20190630175348,LFQ intensity QC5,LFQ intensity QC-ref
0,A0A075B6I0,1748400000,81519000,34828000,64287000,34160000,114300000,0,28089000,514130000,...,50181000,70835000,38138000,0,0,0,0,27537000,0,26435000
1,A0A075B6I9,1449400000,23584000,0,150460000,23808000,40302000,21778000,59043000,153250000,...,251280000,0,35201000,0,0,0,0,0,0,65847000
1,P04211,1449400000,23584000,0,150460000,23808000,40302000,21778000,59043000,153250000,...,251280000,0,35201000,0,0,0,0,0,0,65847000
2,A0A075B6J9,138340000,0,0,0,0,0,0,0,70032000,...,0,0,0,0,0,0,0,0,0,0
3,A0A075B6K4,1927800000,0,0,39734000,0,0,0,0,345740000,...,35047000,27030000,19669000,77345000,62373000,71807000,0,82166000,63249000,85184000


,Protein IDs,Majority protein IDs,Peptide counts (all),Peptide counts (razor+unique),Peptide counts (unique),Protein names,Gene names,Fasta headers,Number of proteins,Peptides,...,LFQ intensity C2-18,LFQ intensity C2-8,LFQ intensity C2-9,LFQ intensity QC_20190614162630,LFQ intensity QC_20190619140322,LFQ intensity QC_20190621214931,LFQ intensity QC_20190629114230,LFQ intensity QC_20190630175348,LFQ intensity QC5,LFQ intensity QC-ref
0,0,A0A0J9YX35;Q5W0Z9;O75264;Q13068;P20963;P25800;...,0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,Q9BUV8,0,0,0,0,0,0,0.0,0.0,...,1056000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,Q99640,0,0,0,0,0,0,0.0,0.0,...,1056000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Q66K79,0,0,0,0,0,0,0.0,0.0,...,1056000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,Q9Y6X9,0,0,0,0,0,0,0.0,0.0,...,1056000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
